In [1]:
import pandas as pd
from pandas import json_normalize
from datetime import date
pd.set_option('display.max_columns', None)

In [2]:
df_author_raw = catalog.load('raw/openalex/author#parquet')

                    INFO     Loading data from raw/openalex/author#parquet (ParquetDataset)...  ]8;id=581265;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=535318;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

# Profiling

In [3]:
df_author_raw.head(3)

,id,orcid,display_name,display_name_alternatives,works_count,cited_by_count,summary_stats,ids,affiliations,last_known_institutions,topics,topic_share,x_concepts,counts_by_year,works_api_url,updated_date,created_date
0,https://openalex.org/A5003447605,https://orcid.org/0000-0002-5260-1807,Gustavo E. Romero,"[Gladis Graciela Romero, Gustavo Romero, Gusta...",760,9045,"{'2yr_mean_citedness': 1.2045454545454546, 'h_...",{'openalex': 'https://openalex.org/A5003447605...,"[{'institution': {'country_code': 'AR', 'displ...","[{'country_code': 'AR', 'display_name': 'Insti...","[{'count': 384, 'display_name': 'High-Energy A...",[{'display_name': 'High-Energy Astrophysics an...,"[{'display_name': 'Physics', 'id': 'https://op...","[{'cited_by_count': 745, 'works_count': 23, 'y...",https://api.openalex.org/works?filter=author.i...,2024-11-27T02:59:56.541657,2023-07-21
1,https://openalex.org/A5053874532,None,Guillermo Soberón,"[Guillermo R Soberon, Guillermo Soberon, G. So...",629,1120,"{'2yr_mean_citedness': 0.0, 'h_index': 14, 'i1...",{'openalex': 'https://openalex.org/A5053874532...,"[{'institution': {'country_code': 'MX', 'displ...","[{'country_code': 'MX', 'display_name': 'Mexfa...","[{'count': 146, 'display_name': 'University So...",[{'display_name': 'Higher Education and Knowle...,"[{'display_name': 'Philosophy', 'id': 'https:/...","[{'cited_by_count': 17, 'works_count': 0, 'yea...",https://api.openalex.org/works?filter=author.i...,2024-11-27T21:11:57.848498,2023-07-21
2,https://openalex.org/A5112709549,None,Hugo Luis López,"[Hugo L. López, Hugo Luis López, Hugo L. Lopez...",480,3300,"{'2yr_mean_citedness': 0.3333333333333333, 'h_...",{'openalex': 'https://openalex.org/A5112709549...,"[{'institution': {'country_code': 'AR', 'displ...","[{'country_code': 'AR', 'display_name': 'Unive...","[{'count': 138, 'display_name': 'Discourse Ana...",[{'display_name': 'Discourse Analysis and Soci...,"[{'display_name': 'Art', 'id': 'https://openal...","[{'cited_by_count': 183, 'works_count': 0, 'ye...",https://api.openalex.org/works?filter=author.i...,2024-11-22T12:03:52.544346,2024-10-15


# Nodo

In [4]:
def land_author2affiliation_openalex(df: pd.DataFrame)-> pd.DataFrame:
    df = df.convert_dtypes()

    # Selecciono columna con id de author y afiliación
    df_author = df.loc[:, ['id', 'affiliations']]

    # Proceso columna 'affiliations'
    df_author = df_author.explode('affiliations').reset_index(drop=True)
    affiliation_expanded = pd.json_normalize(df_author["affiliations"])
    affiliation_expanded = affiliation_expanded.loc[:,['institution.id','years']]

    df_author2affiliation = pd.concat([df_author, affiliation_expanded], axis=1)
    df_author2affiliation.drop(columns=["affiliations"], inplace=True)
    df_author2affiliation = df_author2affiliation.explode('years')
    df_author2affiliation['load_datetime'] = date.today()
    
    return df_author2affiliation



In [5]:
df_author2affiliation = land_author2affiliation_openalex(df_author_raw)

# Resultados

In [6]:
df_author2affiliation

,id,institution.id,years,load_datetime
0,https://openalex.org/A5003447605,https://openalex.org/I151201029,2024,2024-12-03
0,https://openalex.org/A5003447605,https://openalex.org/I151201029,2023,2024-12-03
0,https://openalex.org/A5003447605,https://openalex.org/I151201029,2022,2024-12-03
0,https://openalex.org/A5003447605,https://openalex.org/I151201029,2021,2024-12-03
0,https://openalex.org/A5003447605,https://openalex.org/I151201029,2020,2024-12-03
...,...,...,...,...
11468,https://openalex.org/A5113933053,https://openalex.org/I4210122970,2021,2024-12-03
11469,https://openalex.org/A5114742604,https://openalex.org/I151201029,2024,2024-12-03
11470,https://openalex.org/A5114742604,https://openalex.org/I4210122970,2024,2024-12-03
11471,https://openalex.org/A5114742605,https://openalex.org/I151201029,2024,2024-12-03
